version: 0.0.2.0

## Import das bibliotecas necessárias
    

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import math

import global_
import results
import functions

# **Initial Configuration**

Insert the file's path:

In [2]:
path = "D:/Documentos/Unicamp/IC/Videos/"
folder = ""
name = "basal"
extension = ".avi"

For example:

`path: "C:/Videos/"`

`name: "meu_video"`

`extension: ".avi"`


Set the distance between two pixels of the image **in μm**

In [3]:
d_pixel = 3.5 # Distancia entre pixel = 3.5 um

Set the insect weight in **miligrams**

In [4]:
weight = 120.1 # Peso = 120.1 mg

Determine se a detecção de sístole e diástole será feita através do gráfico da área do coração ou através da área da elipse

| Método de detecção | Descrição |
| --- | --- |
| 1 | Detecção de frequencia cardíaca através da análise da área do contorno do coração |
| 2 | Detecção de frequencia cardíaca através da análise da área da elipse |

In [5]:
detection_method = 1  # 1 = Detecção por area do contorno do coracao

# **Funcionamento**

Depois de configurar as variáveis iniciais na seção acima, execute o notebook.

Serão apresentados duas telas iniciais

The results will be presented at the end of the Notebook at section [Results](#results)

# **Configurações Extras**

Configurações visuais como

font = fonte do texto (para mais fontes consulte o [manual](https://www.oreilly.com/library/view/mastering-opencv-4/9781789344912/16b55e96-1027-4765-85d8-ced8fa071473.xhtml))

In [6]:
font = cv.FONT_HERSHEY_SIMPLEX
font_color = (209, 80, 0, 255)

## **1 Funções Principais**

### Função rectangle ellipse

In [7]:
# Function that apply filters on a image
def filters(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # image to grayscale
    blur = cv.medianBlur(gray,9)    # Apply median blur with kernel = 9
    gauss = cv.GaussianBlur(blur,(9,9),sigmaX=0,sigmaY=0)   # Apply Gaussian blur with kernel = 9
        
    return gauss  

# Adjust the treshold of a image
def treshold_adjustment (args):

    try:
        # Get current treshold and scale from trackbar
        scale = cv.getTrackbarPos('Scale','Comandos')
        treshold = cv.getTrackbarPos('Treshold','Comandos')
    except:
        scale = 100
        treshold = 254

    # Make copy of unaltered frame
    dias_frame = np.copy(max_frame)
    sys_frame = np.copy(min_frame)

    # Apply filters on both frames
    dias_filter  = filters(dias_frame)
    sys_filter  = filters(sys_frame)
    _ , dias_binary_img = cv.threshold(dias_filter,treshold,255,cv.THRESH_BINARY)
    _ , sys_binary_img = cv.threshold(sys_filter,treshold,255,cv.THRESH_BINARY)

    # Detect binary edges
    dias_binary_edges = cv.Canny(dias_binary_img, 50, 100, 5, L2gradient= True)
    sys_binary_edges = cv.Canny(sys_binary_img, 50, 100, 5, L2gradient= True)

    # Find contours on both frames
    dias_contours,_ = cv.findContours(dias_binary_edges, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    sys_contours,_ = cv.findContours(sys_binary_edges, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    # Detect greatest contour of both frames and get its index
    max_area = -1
    for i, c in enumerate(sys_contours):
        if cv.arcLength(c,True) > max_area: 
            max_area = cv.arcLength(c,True)
            max_i_sys = i

    max_area = -1
    for i, c in enumerate(dias_contours):
        if cv.arcLength(c,True) > max_area: 
            max_area = cv.arcLength(c,True)
            max_i_dias = i

    # Apply scale to both contours (Systole and Diastole)
    scaled_dias = scale_contour(dias_contours[max_i_dias],scale/100)
    scaled_sys = scale_contour(sys_contours[max_i_sys],scale/100)
    
    # Draw contours on both frames
    cv.drawContours(dias_frame,scaled_dias,-1,(255,0,255),1)
    cv.drawContours(sys_frame,scaled_sys,-1,(255,0,255),1)

    # Obtain ellipse param as  (center x,center y), (height,width), ângle
    min_center,min_dim,min_angle = cv.fitEllipseDirect(scaled_sys) 
    max_center,max_dim,max_angle = cv.fitEllipseDirect(scaled_dias)
       
    avg_height = (max_dim[1] + min_dim[1])/2

    max_ellipse = (max_center,(max_dim[0],avg_height),0)
    min_ellipse = (min_center,(min_dim[0],avg_height),0) 

    # Draw ellipses on both frames
    cv.ellipse(dias_frame, max_ellipse, (255,0,255), 1)
    cv.ellipse(sys_frame, min_ellipse, (255,0,255), 1)

    global ellipse_vertex
    ellipse_vertex = np.zeros((4,2))

    dias_box = rectangle_ellipse(max_ellipse)
    dias_param = ellipse_vertex

    ellipse_vertex = np.zeros((4,2))

    sys_box = rectangle_ellipse(min_ellipse)
    sys_param = ellipse_vertex
       
    del ellipse_vertex

    # Show both frames with ellipse adjusted
    cv.imshow('Systole',sys_frame)
    cv.imshow('Diastole',dias_frame)
    
    return

def get_filters_parameters (file):

    treshold = 255
    video = cv.VideoCapture(file)   # Opens video       
    global max_frame, min_frame, first_frame, frame_height, frame_width

    ret, first_frame = video.read()
    gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    _, bw_img = cv.threshold(gray,160,255,cv.THRESH_BINARY)
    wht_px = np.count_nonzero(bw_img)

    frame_height = first_frame.shape[0]
    frame_width = first_frame.shape[1]

    count = 0
    min_frame = first_frame
    max_frame = first_frame

    max_wht_px = wht_px
    min_wht_px = wht_px

    while (count <= video.get(cv.CAP_PROP_FRAME_COUNT)//3):

      ret, img = video.read()
      gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      _, bw_img = cv.threshold(gray,160,255,cv.THRESH_BINARY)

      wht_px = np.count_nonzero(bw_img)

      if wht_px > max_wht_px:
        max_frame = img
        max_wht_px = wht_px
        max_time = video.get(cv.CAP_PROP_POS_MSEC)/1000
      elif wht_px < min_wht_px:
        min_frame = img
        min_wht_px = wht_px
        min_time = video.get(cv.CAP_PROP_POS_MSEC)/1000

      count = count + 1

    # Show images on systole and diastole for trehsold adjustment
    cv.imshow('Systole',min_frame)
    cv.imshow('Diastole',max_frame)

    # Create window with 3 trackbar
    cv.namedWindow('Comandos', cv.WINDOW_NORMAL)
    cv.resizeWindow('Comandos', 700, 100)
    cv.createTrackbar('Treshold','Comandos',254,254,treshold_adjustment)
    cv.createTrackbar('Axis','Comandos',0,frame_width,axis_adjustment)
    cv.createTrackbar('Scale','Comandos',100,100,treshold_adjustment)

    # Waits for the user to press 'n' key
    key = cv.waitKey(0)
    while (key != ord('n')):
        key = cv.waitKey(0)
    
    # Get values and close windows
    treshold = cv.getTrackbarPos('Treshold','Comandos')
    axis_location = cv.getTrackbarPos('Axis','Comandos')
    scale = cv.getTrackbarPos('Scale','Comandos')/100

    cv.destroyAllWindows()   

    return treshold, axis_location, scale

In [8]:
def rectangle_ellipse(parameters):
    from global_ import ellipse_vertex  # Get current value for global var

    # List contaning the 4 vertex of rectangle that fits the parameter
    box2 = cv.boxPoints(parameters)
    box2 = np.intp(box2)     
    
    # Inferior ellipse vertex  
    global_.ellipse_vertex[0][0] = (box2[0][0] + box2[3][0])//2
    global_.ellipse_vertex[0][1] = (box2[0][1] + box2[3][1])//2

    # Superior ellipse vertex
    global_.ellipse_vertex[1][0] = (box2[1][0] + box2[2][0])//2
    global_.ellipse_vertex[1][1] = (box2[1][1] + box2[2][1])//2

    # Right ellipse vertex
    global_.ellipse_vertex[2][0] = (box2[2][0] + box2[3][0])//2
    global_.ellipse_vertex[2][1] = (box2[2][1] + box2[3][1])//2
            
    # Left ellipse vertex
    global_.ellipse_vertex[3][0] = (box2[0][0] + box2[1][0])//2
    global_.ellipse_vertex[3][1] = (box2[0][1] + box2[1][1])//2        

    global_.ellipse_vertex = ellipse_vertex.astype(int)
    
    return box2

In [9]:
def get_ellipse_param():
    from global_ import ellipse_vertex
    global ellipse_center, ellipse_height, ellipse_width, ellipse_size #ellipse_vertex
    
    # Calcula o centro da elipse
    ellipse_center = ((global_.ellipse_vertex[2][0] + global_.ellipse_vertex[3][0])//2,(global_.ellipse_vertex[0][1] + global_.ellipse_vertex[1][1])//2)

    ellipse_height = int(distance(global_.ellipse_vertex[0][0],global_.ellipse_vertex[0][1],
                            global_.ellipse_vertex[1][0],global_.ellipse_vertex[1][1]))
    ellipse_width = int(distance(global_.ellipse_vertex[2][0],global_.ellipse_vertex[2][1],
                            global_.ellipse_vertex[3][0],global_.ellipse_vertex[3][1]))
    ellipse_size = (ellipse_width,ellipse_height)

    return

###  Função fit_ellipse

In [10]:
def fit_ellipse(source_img,method,join,stop,scale):

    # Fit the contour that fits the heart shape
    cnts,_ = cv.findContours(source_img, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    #aux_plot_points(contours)

    # Matrizes que guardam a area, pontos do retangulo e elipse de cada contorno
    #minEllipse = [None]*len(cnts)
    center_contours = []
    
    best_i = 0
    
    max_area = -1
    
    # Loop que obtem a altura, largura e altura de cada contorno. 
    # Salva os parametros do contorno de maior area (parte central do coração)
    for i, c in enumerate(cnts):

        # Encontra as coordenadas do retângulo que circunscreve o contorno "c"
        top_left_x,top_left_y,largura,altura = cv.boundingRect(c)

        # Calcula o centro do retângulo
        centerx = (2*top_left_x + largura)/2
        centery = (2*top_left_y + altura)/2
        center_contours.append([centerx,centery])

        # Encontra o contorno de maior area 
        if cv.contourArea(c) > max_area: 
            max_area = cv.contourArea(c)
            estimated_centerx = centerx
            estimated_centery = centery
            best_i = i
    
    # Se distância maior que 10, então a divisão aparente será removida
    if distance(estimated_centerx,estimated_centery,avg_x_center,avg_y_center) > 20 or join :
        #contours = remove_division(contours, best_i)
        if method == True:
            method = False
        else:
            stop = True
    
    max_area = -1

    for i, c in enumerate(cnts):

        if cv.contourArea(c) > max_area: 
            max_area = cv.contourArea(c)
            main_cnt = i

    
    heart_cnt = cnts[main_cnt]
    scaled_heart_cnt = scale_contour(heart_cnt,scale)

    minEllipse = cv.fitEllipseDirect(scaled_heart_cnt)

    top_left_x,top_left_y,largura,altura = cv.boundingRect(scaled_heart_cnt)

    centerx = (2*top_left_x + largura)/2
    centery = (2*top_left_y + altura)/2

    center_contours.append([float(centerx),float(centery)])

    estimated_area = cv.contourArea(scaled_heart_cnt)         
    estimeated_height = altura
    estimated_width = largura
    estimated_top_left_x = top_left_x
    estimated_top_left_y = top_left_y
    estimated_centerx = centerx
    estimated_centery = centery

    drawing_img = np.zeros((source_img.shape[0], source_img.shape[1], 3), dtype=np.uint8)
    
    for i, c in enumerate(cnts):
        color = (0,255,0)
        
        if cv.contourArea(c) == max_area: 

            #aux_plot_points(contours)
            
            # Obtem parametros da elipse ((centrox,centroy), (altura,largura), ângulo)
            ellipse_param =  minEllipse

            global ellipse_angle
            ellipse_angle = ellipse_param[2]

            # Desenha o contorno principal e a elipse circunscrita
            cv.drawContours(drawing_img, scaled_heart_cnt, -1,color)
            cv.ellipse(drawing_img, minEllipse, (0,255,255), 1)
            #cv.ellipse(frame, minEllipse[i], (0,0,255), 1)

            #global ellipse_vertex 
            from global_ import ellipse_vertex
            global_.ellipse_vertex = np.zeros((4,2))

            box2 = rectangle_ellipse(minEllipse)

            get_ellipse_param()

            # Desenha os vértices da elipse        
            cv.circle(drawing_img,(global_.ellipse_vertex[0][0],global_.ellipse_vertex[0][1]),0,(0,255,255),8)
            cv.circle(drawing_img,(global_.ellipse_vertex[1][0],global_.ellipse_vertex[1][1]),0,(0,255,255),8)
            cv.circle(drawing_img,(global_.ellipse_vertex[2][0],global_.ellipse_vertex[2][1]),0,(255,255,255),8)
            cv.circle(drawing_img,(global_.ellipse_vertex[3][0],global_.ellipse_vertex[3][1]),0,(0,0,255),8)     
        else:
            # Desenha os outros contornos secundários
            cv.drawContours(drawing_img, cnts, i, (255,255,255))

    cv.rectangle(drawing_img,(estimated_top_left_x,estimated_top_left_y),
                (estimated_top_left_x+estimated_width,estimated_top_left_y+estimeated_height),(0,255,0),2)

    return scaled_heart_cnt, main_cnt, ellipse_param, box2 ,drawing_img,estimated_area,method,stop

In [11]:
def get_vol_rev(cont,axis):

    right_cont_x = np.array([])
    right_cont_y = np.array([])
    left_cont_x = np.array([])
    left_cont_y = np.array([])
    fx2 = np.array([])

    for v in range(len(cont)):
        x_cont = cont[v][0][0]
        y_cont = cont[v][0][1]

        if x_cont > axis:
            right_cont_x = np.append(right_cont_x,x_cont)
            right_cont_y = np.append(right_cont_y,y_cont)
        else:
            left_cont_x = np.append(left_cont_x,x_cont)
            left_cont_y = np.append(left_cont_y,y_cont)

    a = int(np.amin(right_cont_y) )
    b = int(np.amax(right_cont_y) )
    
    fx = get_fx(right_cont_x,right_cont_y,axis) 

    fx2 = fx**2

    v1 = midpoint(fx2,a,b)/2
    a1 = midpoint(fx,a,b)

    a = int(np.amin(left_cont_y) )
    b = int(np.amax(left_cont_y) )
    
    fx = get_fx(left_cont_x,left_cont_y,axis)

    fx2 = fx**2

    v2 = midpoint(fx2,a,b)/2
    a2 = midpoint(fx,a,b)

    v = v1 + v2
    a = a1 + a2

    return 3.5*3.5*3.5*np.pi*v, 3.5*3.5*a

In [12]:
def midpoint(f, a, b):
    #h = int((b-a)/n)
    h = 1
    result = 0
    #or i in range(int(b-a)):
        #result += f[int(a + h/2) + i*h]
        #esult += f[int(a + h) + i*h]
    #esult *= h
    
    result = np.sum(f)
    
    return result

In [13]:
def get_fx(x,y,axis):

    fx = np.array([])

    a = np.amin(y) 
    b = np.amax(y) 

    # Sort x and y
    p = y.argsort()
    y = y[p]
    x = x[p]

    values, un_index ,counts = np.unique(y,return_index=True,return_counts=True)
    
    counts = counts - 1
    
    for i in range(len(counts)):
        if counts[i] == 0:
            fx =  np.append(fx, x[un_index[i]])
        else:
            fx = np.append( fx, np.average( x [ un_index[i] : un_index[i] + counts[i] ]  ) )
    
    if np.average(fx) < axis:
        return axis - fx
    else:
        return fx - axis

###  Função get_parameters

In [14]:
def get_paramaters(d_pixel):         
    from global_ import ellipse_vertex

    # Calcula o valor da altura e largura da elipse
    a_value = np.sqrt((global_.ellipse_vertex[0][0] - global_.ellipse_vertex[1][0])**2 + (global_.ellipse_vertex[0][1] - global_.ellipse_vertex[1][1])**2)
    b_value = np.sqrt((global_.ellipse_vertex[2][0] - global_.ellipse_vertex[3][0])**2 + (global_.ellipse_vertex[2][1] - global_.ellipse_vertex[3][1])**2)
            
    # Calcula o valor da area e volume da elipse
    area = a_value*math.pi*b_value
    volume = int((1/6)*np.pi*a_value*b_value**2) 

    return int(a_value*d_pixel), int(b_value*d_pixel), int(area*d_pixel**2) ,int(volume*d_pixel**3)

In [15]:
def get_paramaters2(d_px,c,x_ax,y_ax):

    left_x = np.array([])
    right_x = np.array([])

    interval = 5

    for i in range (len(c)):
        if abs(c[i][0][1] - y_ax) <= 5:
            if c[i][0][0] <= x_ax:
                left_x = np.append(left_x,x_ax - c[i][0][0])
            else:
                right_x = np.append(right_x,c[i][0][0] - x_ax)

    topmost = tuple(c[c[:,:,1].argmin()][0])
    bottommost = tuple(c[c[:,:,1].argmax()][0])

    b = np.average(left_x) + np.average(right_x)
    a = bottommost[1] - topmost[1]
   
    volume = int((1/6)*np.pi*a*b**2) 

    return int(a*d_px), int(b*d_px) ,int(volume*d_px**3)

###  Função find_largest_contour

In [16]:
def find_largest_contour (cont,centers,elipses):
    for i, c in enumerate(cont):

        top_left_x,top_left_y,largura,altura = cv.boundingRect(c)

        centerx = (2*top_left_x + largura)//2
        centery = (2*top_left_y + altura)//2

        centers.append([centerx,centery])

        if cv.contourArea(c) > max_area: 
            max_area = cv.contourArea(c)
            estimated_area = max_area
            estimeated_height = altura
            estimated_width = largura
            estimated_top_left_x = top_left_x
            estimated_top_left_y = top_left_y
            estimated_centerx = centerx
            estimated_centery = centery
            best_i = i

            try:
                elipses[i] = cv.fitEllipseDirect(c)
            except:
                None

###  Função get_filters_paramaters

In [17]:
# Function that apply filters on a image
def filters(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # image to grayscale
    blur = cv.medianBlur(gray,9)    # Apply median blur with kernel = 9
    gauss = cv.GaussianBlur(blur,(9,9),sigmaX=0,sigmaY=0)   # Apply Gaussian blur with kernel = 9
        
    return gauss

def get_filters_parameters (file):

    treshold = 255

    # Faz a leitura do primeiro frame do video
    video = cv.VideoCapture(file)       
    global max_frame, min_frame, first_frame, frame_height, frame_width

    ret, first_frame = video.read()
    gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    _, bw_img = cv.threshold(gray,160,255,cv.THRESH_BINARY)
    wht_px = np.count_nonzero(bw_img)

    frame_height = first_frame.shape[0]
    frame_width = first_frame.shape[1]

    count = 0
    min_frame = first_frame
    max_frame = first_frame

    max_wht_px = wht_px
    min_wht_px = wht_px

    while (count <= video.get(cv.CAP_PROP_FRAME_COUNT)//3):

      ret, img = video.read()
      gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      _, bw_img = cv.threshold(gray,160,255,cv.THRESH_BINARY)

      wht_px = np.count_nonzero(bw_img)

      if wht_px > max_wht_px:
        max_frame = img
        max_wht_px = wht_px
        max_time = video.get(cv.CAP_PROP_POS_MSEC)/1000
      elif wht_px < min_wht_px:
        min_frame = img
        min_wht_px = wht_px
        min_time = video.get(cv.CAP_PROP_POS_MSEC)/1000

      count = count + 1

    # Show images on systole and diastole for trehsold adjustment
    cv.imshow('Systole',min_frame)
    cv.imshow('Diastole',max_frame)

    # Create window with 3 trackbar
    cv.namedWindow('Comandos', cv.WINDOW_NORMAL)
    cv.resizeWindow('Comandos', 700, 100)
    cv.createTrackbar('Treshold','Comandos',254,254,treshold_adjustment)
    cv.createTrackbar('Axis','Comandos',0,frame_width,ver_axis_adjustment)
    cv.createTrackbar('Diameter','Comandos',0,frame_height,hor_axis_adjustment)
    cv.createTrackbar('Scale','Comandos',100,100,treshold_adjustment)

    # Waits for the user to press 'n' key
    key = cv.waitKey(0)
    while (key != ord('n')):
        key = cv.waitKey(0)
    
    # Get values and close windows
    treshold = cv.getTrackbarPos('Treshold','Comandos')
    axis_location = cv.getTrackbarPos('Axis','Comandos')
    scale = cv.getTrackbarPos('Scale','Comandos')/100
    hor_axis = cv.getTrackbarPos('Diameter','Comandos')

    cv.destroyAllWindows()   

    return treshold, axis_location, scale, hor_axis

In [18]:
def scale_contour(cnt, scale):
    M = cv.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx,0]
    #cnt_scaled = cnt_norm * scale
    
    cnt_scaled = np.copy(cnt_norm)

    for i in range(len(cnt_norm)):
        cnt_scaled[i][0][0] = cnt_norm[i][0][0]*scale
    
    cnt_scaled = cnt_scaled + [cx,0]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

In [19]:
def ver_axis_adjustment(args):
    dias_frame = np.copy(max_frame)
    sys_frame = np.copy(min_frame)

    cv.line(dias_frame,(args,0),(args,frame_height),(255,0,255),1)
    cv.line(sys_frame,(args,0),(args,frame_height),(255,0,255),1)

    cv.imshow('Systole',sys_frame)
    cv.imshow('Diastole',dias_frame)

    return

In [20]:
def hor_axis_adjustment(args):
    dias_frame = np.copy(max_frame)
    sys_frame = np.copy(min_frame)

    cv.line(dias_frame,(0,args),(frame_width,args),(255,0,255),1)
    cv.line(sys_frame,(0,args),(frame_width,args),(255,0,255),1)

    cv.imshow('Systole',sys_frame)
    cv.imshow('Diastole',dias_frame)

    return

In [21]:
def treshold_adjustment (args):

    try:
        # Get current treshold and scale from trackbar
        scale = cv.getTrackbarPos('Scale','Comandos')
        treshold = cv.getTrackbarPos('Treshold','Comandos')
    except:
        scale = 100
        treshold = 254

    # Make copy of unaltered frame
    dias_frame = np.copy(max_frame)
    sys_frame = np.copy(min_frame)

    # Apply filters on both frames
    dias_filter  = filters(dias_frame)
    sys_filter  = filters(sys_frame)
    _ , dias_binary_img = cv.threshold(dias_filter,treshold,255,cv.THRESH_BINARY)
    _ , sys_binary_img = cv.threshold(sys_filter,treshold,255,cv.THRESH_BINARY)

    # Detect binary edges
    dias_binary_edges = cv.Canny(dias_binary_img, 50, 100, 5, L2gradient= True)
    sys_binary_edges = cv.Canny(sys_binary_img, 50, 100, 5, L2gradient= True)

    # Find contours on both frames
    dias_contours,_ = cv.findContours(dias_binary_edges, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    sys_contours,_ = cv.findContours(sys_binary_edges, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    # Detect greatest contour of both frames and get its index
    max_area = -1
    for i, c in enumerate(sys_contours):
        if cv.arcLength(c,True) > max_area: 
            max_area = cv.arcLength(c,True)
            max_i_sys = i

    max_area = -1
    for i, c in enumerate(dias_contours):
        if cv.arcLength(c,True) > max_area: 
            max_area = cv.arcLength(c,True)
            max_i_dias = i

    # Apply scale to both contours (Systole and Diastole)
    scaled_dias = scale_contour(dias_contours[max_i_dias],scale/100)
    scaled_sys = scale_contour(sys_contours[max_i_sys],scale/100)
    
    # Draw contours on both frames
    cv.drawContours(dias_frame,scaled_dias,-1,(255,0,255),1)
    cv.drawContours(sys_frame,scaled_sys,-1,(255,0,255),1)

    # Obtain ellipse param as  (center x,center y), (height,width), ângle
    min_center,min_dim,min_angle = cv.fitEllipseDirect(scaled_sys) 
    max_center,max_dim,max_angle = cv.fitEllipseDirect(scaled_dias)
       
    avg_height = (max_dim[1] + min_dim[1])/2

    max_ellipse = (max_center,(max_dim[0],avg_height),0)
    min_ellipse = (min_center,(min_dim[0],avg_height),0) 

    # Draw ellipses on both frames
    #cv.ellipse(dias_frame, max_ellipse, (255,0,255), 1)
    #cv.ellipse(sys_frame, min_ellipse, (255,0,255), 1)

    # Draw ellipses on both frames
    cv.ellipse(dias_frame, (max_center,max_dim,max_angle), (255,0,255), 1)
    cv.ellipse(sys_frame, (min_center,min_dim,min_angle), (255,0,255), 1)

    global ellipse_vertex
    ellipse_vertex = np.zeros((4,2))

    dias_box = rectangle_ellipse(max_ellipse)
    dias_param = ellipse_vertex

    ellipse_vertex = np.zeros((4,2))

    sys_box = rectangle_ellipse(min_ellipse)
    sys_param = ellipse_vertex
       
    del ellipse_vertex

    # Show both frames with ellipse adjusted
    cv.imshow('Systole',sys_frame)
    cv.imshow('Diastole',dias_frame)
    
    return

###  Função get_avarage_center

In [22]:
def get_average_center(file,treshold):

    # Faz a abertura do video
    video = cv.VideoCapture(file)
    
    # Inicializacao das variaveis
    n_frames = 0
    ret = True
    x_sum = 0
    y_sum = 0
    
    # Enquanto o vídeo estiver aberto, a cada frame do video obtemos a posicao central do retangulo
    # que envolve o contorno do coracao e no fim e calculado a media dos valores de centro
    while video.isOpened():

        # faz a leitura do frame 
        ret, frame = video.read()

        # testa se o frame foi aberto (ret == True)        
        if ret:

            max_area = -1   # Inicializa a area maxima para o frame atual

            # Aplica os filtros no frame
            filtered = filters(frame)
            _ , binary_img = cv.threshold(filtered,treshold,250,cv.THRESH_BINARY)
            
            # Encontra os contornos da imagem binaria
            contours,_ = cv.findContours(binary_img, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)
        
            # Obtem o centro do retangulo que engloba o maior contorno
            for i, c in enumerate(contours):

                if cv.contourArea(c) > max_area: 
                    centerx,centery,largura,altura = cv.boundingRect(c)
                    max_area = cv.contourArea(c)

            x_sum = x_sum + (2*centerx + largura)//2
            y_sum = y_sum + (2*centery + altura)//2

            n_frames = n_frames + 1 
        
        else:
            break
    
    video.release() #libera o video
    del frame
    del contours
    del video
    
    return x_sum//n_frames, y_sum//n_frames

In [23]:
def remove_division(cont,main_i):

    joined_cont = cont
    joined_i = list()

    x_main_cont = cont[main_i][0][0][0]
    y_main_cont = cont[main_i][0][0][1]

    tmp_draw = np.zeros((frame.shape[0], frame.shape[1], 4))
    try_i = 0
    min_dist = 0
    
    removed = False

    while True:
        cv.imshow('Juntar', tmp_draw)

        cmd = cv.waitKey(0)

        if cmd == ord('n'):

            cont_temp = joined_cont
            
            tmp_draw = np.zeros((frame.shape[0], frame.shape[1], 4))

            try_i = (try_i + 1)%len(cont)

            if (try_i == main_i):
                try_i = (try_i + 1)%len(cont)

            s = set(joined_i)

            if (len(joined_cont)!=0) and (try_i in s):
                try_i = (try_i + 1)%len(cont)

            
            cont_temp = join_countours(cont_temp,main_i,try_i)

            cv.drawContours(tmp_draw, cont_temp, try_i, (255,255,255))
            cv.drawContours(tmp_draw, cont, main_i, (0,255,255))
            
            removed = True
            
        elif cmd == ord('j') and removed:
            joined_cont = cont_temp
            joined_i.append(try_i)
            tmp_draw = np.zeros((frame.shape[0], frame.shape[1], 4))
            cv.drawContours(tmp_draw, cont_temp, main_i, (0,255,255))

        elif cmd == ord('c') and removed:
            cv.destroyWindow('Juntar')
            break

        elif cmd == ord('s'):
            cv.destroyWindow('Juntar')        
            cont_temp = cont
            break              



    return cont_temp

In [24]:
def join_countours(c,main_i,min_i):

    c = np.copy(c)
    
    x_main_cont = c[main_i][0][0][0]
    y_main_cont = c[main_i][0][0][1]

    x_min_cont = c[min_i][0][0][0]
    y_min_cont = c[min_i][0][0][1]

    min_lenght = c[min_i].shape[0]
    main_lenght = c[main_i].shape[0]

    if y_min_cont < y_main_cont :
        for i in range(len(c[min_i])):

            x_cont = c[min_i][i][0][0]
            y_cont = c[min_i][i][0][1]

            dist = distance(x_cont,y_cont,x_main_cont,y_main_cont)

            if i == 0:
                min_dist = dist
                cont_i = i
            else:
                if (dist < min_dist):
                    cont_i = i
                    min_dist = dist

        for i in range(len(c[main_i])):
            c[min_i] = np.insert(c[min_i], 2*cont_i + 2*(i + 1), [c[main_i][i][0][0],c[main_i][i][0][1]])
        
        c[min_i] = np.reshape(c[min_i],(min_lenght + main_lenght,1,2))
        
    elif y_min_cont > y_main_cont:
        for i in range(len(c[main_i])):

            x_cont = c[main_i][i][0][0]
            y_cont = c[main_i][i][0][1]

            dist = distance(x_cont,y_cont,x_min_cont,y_min_cont)

            if i == 0:
                min_dist = dist
                cont_i = i
            else:
                if (dist < min_dist):
                    cont_i = i
                    min_dist = dist
        for i in range(len(c[min_i])):
            c[main_i] = np.insert(c[main_i], 2*cont_i + 2*(i + 1), [c[min_i][i][0][0],c[min_i][i][0][1]])

        c[main_i] = np.reshape(c[main_i],(min_lenght + main_lenght,1,2))
    return c

###  Função ellipse_rotation

In [25]:
def ellipse_rotation(args):
    global ellipse_angle, rec
    global ellipse_center, ellipse_height, ellipse_width, ellipse_size
    global ellipse_vertex

    drawing = np.copy(cache)

    ellipse_angle = args

    rec = rectangle_ellipse((ellipse_center,ellipse_size,ellipse_angle))

    cv.circle(drawing,(ellipse_vertex[0][0],ellipse_vertex[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
    cv.circle(drawing,(ellipse_vertex[1][0],ellipse_vertex[1][1]),0,(0,0,255),8) # Ponto de cima da elipse
    cv.circle(drawing,(ellipse_vertex[2][0],ellipse_vertex[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
    cv.circle(drawing,(ellipse_vertex[3][0],ellipse_vertex[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elips
    cv.circle(drawing,(ellipse_center[0],ellipse_center[1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

    cv.ellipse(drawing,(ellipse_center,ellipse_size,ellipse_angle), (90,90,255), 1)

    cv.imshow('Original',drawing)   
    
    return

###  Função video_progression

In [26]:
def video_progression(args):

    global skip
    
    if key != ord('n') or key != ord('n'):

        next_frame = video.get(cv.CAP_PROP_POS_FRAMES)
        current_frame = next_frame - 1
        previous_frame = current_frame - 1

        if args < current_frame:
            if previous_frame >= 0:
                video.set(cv.CAP_PROP_POS_FRAMES, previous_frame)
            else:
                previous_frame = 0
    
        skip = True

    return


###  Função modify_ellipse

In [27]:
def modify_elipse(action, x, y, flags, parameters):
    
    frame = np.copy(parameters[0])
    #rec = parameters[1]
    cache = np.copy(parameters[2])

    global edit
    # Declara o vetor com os pontos da elipse como global
    global ellipse_vertex, ellipse_angle, rec

    global ellipse_center, ellipse_height, ellipse_width, ellipse_size

    
    if action == cv.EVENT_LBUTTONDOWN:

        cv.ellipse(cache,(ellipse_center,ellipse_size,ellipse_angle), (0,0,255), 1)
        
        if (((x> ellipse_vertex[3][0]) and (x<ellipse_vertex[2][0]) and (y>ellipse_vertex[1][1]) and (y< ellipse_vertex[0][1])
            and (ellipse_angle <=90)) or ((x< ellipse_vertex[3][0]) and (x>ellipse_vertex[2][0]) and (y<ellipse_vertex[1][1]) 
            and (y> ellipse_vertex[0][1]) and (ellipse_angle >90))) :

            cv.circle(cache,(ellipse_vertex[0][0],ellipse_vertex[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
            cv.circle(cache,(ellipse_vertex[1][0],ellipse_vertex[1][1]),0,(0,0,255),8) # Ponto de cima da elipse
            cv.circle(cache,(ellipse_vertex[2][0],ellipse_vertex[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
            cv.circle(cache,(ellipse_vertex[3][0],ellipse_vertex[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse
            cv.circle(cache,(ellipse_center[0],ellipse_center[1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

            cv.circle(cache,(rec[0][0],rec[0][1]),0,(0,0,255),8) # Canto inferior esquerdo do retângulo
            cv.circle(cache,(rec[1][0],rec[1][1]),0,(0,0,255),8) # Canto superior esquerdo do retângulo
            cv.circle(cache,(rec[2][0],rec[2][1]),0,(0,0,255),8) # Canto superior direito do retângulo
            cv.circle(cache,(rec[3][0],rec[3][1]),0,(0,0,255),8) # Canto inferior direito do retângulo
            # Desenha o retângulo que envolve a elipse
            box = np.array([[rec[0][0],rec[0][1]],[rec[1][0],rec[1][1]],[rec[2][0],rec[2][1]],[rec[3][0],rec[3][1]]])
            cv.drawContours(cache,[box],0,(0,0,255),1)
            cv.imshow('Original',cache)
            edit = True
        else:
            cv.imshow('Original',cache)
            edit = False


    elif (action == cv.EVENT_MOUSEMOVE) and (flags == cv.EVENT_FLAG_LBUTTON):

        #cv.circle(cache,(vertex_ellipse[0][0],vertex_ellipse[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
        #cv.circle(cache,(vertex_ellipse[1][0],vertex_ellipse[1][1]),0,(0,255,255),8) # Ponto de cima da elipse
        #cv.circle(cache,(vertex_ellipse[2][0],vertex_ellipse[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
        #cv.circle(cache,(vertex_ellipse[3][0],vertex_ellipse[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

        #cv.circle(cache,(rec[0][0],rec[0][1]),0,(0,0,255),8) # Canto inferior esquerdo do retângulo
        #cv.circle(cache,(rec[1][0],rec[1][1]),0,(0,0,255),8) # Canto superior esquerdo do retângulo
        #cv.circle(cache,(rec[2][0],rec[2][1]),0,(0,0,255),8) # Canto superior direito do retângulo
        #cv.circle(cache,(rec[3][0],rec[3][1]),0,(0,0,255),8) # Canto inferior direito do retângulo

        
        #cv.rectangle(cache,(rec[0][0],rec[0][1]),(rec[2][0],rec[2][1]),(0,255,0),1)

        # Redimensionamento da parte de baixo da elipse
        if distance(ellipse_vertex[0][0],ellipse_vertex[0][1],x,y) < 10:
            dif = ellipse_vertex[0][1] - y
            ellipse_vertex[0][1] =  y
            rec[0][1] -= dif
            rec[3][1] -= dif            
        
        # Redimensionamento da parte de cima da elipse
        elif distance(ellipse_vertex[1][0],ellipse_vertex[1][1],x,y) < 10:
            dif = ellipse_vertex[1][1] - y
            ellipse_vertex[1][1] =  y
            rec[1][1] -= dif
            rec[2][1] -= dif

        # Redimensionamento da parte lateral direita da elipse
        elif distance(ellipse_vertex[2][0],ellipse_vertex[2][1],x,y) < 10:
            dif = ellipse_vertex[2][0] - x
            ellipse_vertex[2][0] =  x
            rec[2][0] -= dif
            rec[3][0] -= dif
        
        # Redimensionamento da parte lateral esquerdo da elipse
        elif distance(ellipse_vertex[3][0],ellipse_vertex[3][1],x,y) < 10:
            dif = ellipse_vertex[3][0] - x
            ellipse_vertex[3][0] =  x
            rec[0][0] -= dif
            rec[1][0] -= dif


        # Move a elipse
        elif distance(ellipse_center[0],ellipse_center[1],x,y) < 10:
            difx = ellipse_center[0] - x
            dify = ellipse_center[1] - y
            #ellipse_vertex[3][0] =  - x
            rec[0][0] -= difx
            rec[1][0] -= difx
            rec[2][0] -= difx
            rec[3][0] -= difx


            rec[1][1] -= dify
            rec[2][1] -= dify
            rec[0][1] -= dify
            rec[3][1] -= dify            

        #get_ellipse_param()
        #rectangle_ellipse((ellipse_center,ellipse_size,ellipse_angle))

        # Calcula a posição do ponto lateral direito e esquerdo da elipse
        ellipse_vertex[2][1] = (rec[2][1] + rec[3][1])//2
        ellipse_vertex[3][1] = (rec[0][1] + rec[1][1])//2
        
        # Calcula a posição do ponto superior e inferior da elipse
        ellipse_vertex[1][0] = (rec[1][0] + rec[2][0])//2
        ellipse_vertex[0][0] = (rec[0][0] + rec[3][0])//2

        ellipse_vertex[2][0] = (rec[2][0] + rec[3][0])//2
        ellipse_vertex[3][0] = (rec[0][0] + rec[1][0])//2
        ellipse_vertex[1][1] = (rec[1][1] + rec[2][1])//2
        ellipse_vertex[0][1] = (rec[0][1] + rec[3][1])//2

        get_ellipse_param()

        # Desenha o retângulo que envolve a elipse
        #box = np.array([[rec[0][0],rec[0][1]],[rec[1][0],rec[1][1]],[rec[2][0],rec[2][1]],[rec[3][0],rec[3][1]]])
        #cv.drawContours(cache,[box],0,(0,0,255),1)

        # Desenha a elipse
        cv.ellipse(cache,(ellipse_center,ellipse_size,ellipse_angle), (90,90,255), 1)

        #cv.ellipse(cache, box, color = (0, 0, 255), thickness = 1)

        cv.imshow('Original',cache)

        edit = True

        #height,width,volume = get_paramaters(d_pixel)

        #partial_info  = np.copy(info_frame)
        #cv.putText(partial_info,"Volume: " + str(int(volume)/1e+9) + " ul", (30,70),font ,1/2,font_color, 2) 
        #cv.imshow('Informacao',partial_info)

        #cv.circle(frame,(x,y),0,(0,255,255),8)
    
    elif (action == cv.EVENT_LBUTTONUP):

        if edit == True:
            print(3.5*math.sqrt((ellipse_vertex[0][0] - ellipse_vertex[1][0])**2 + (ellipse_vertex[0][1] - ellipse_vertex[1][1])**2))
            edit = False
            stop = False
            try:

                rec = rectangle_ellipse((ellipse_center,ellipse_size,ellipse_angle))
                
                cv.circle(cache,(ellipse_vertex[0][0],ellipse_vertex[0][1]),0,(0,0,255),8) # Ponto de baixo da elipse
                cv.circle(cache,(ellipse_vertex[1][0],ellipse_vertex[1][1]),0,(0,0,255),8) # Ponto de cima da elipse
                cv.circle(cache,(ellipse_vertex[2][0],ellipse_vertex[2][1]),0,(0,0,255),8) # Ponto lateral direito da elipse
                cv.circle(cache,(ellipse_vertex[3][0],ellipse_vertex[3][1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse
                cv.circle(cache,(ellipse_center[0],ellipse_center[1]),0,(0,0,255),8) # Ponto lateral esquerdo da elipse

                cv.circle(cache,(rec[0][0],rec[0][1]),0,(0,0,255),8) # Canto inferior esquerdo do retângulo
                cv.circle(cache,(rec[1][0],rec[1][1]),0,(0,0,255),8) # Canto superior esquerdo do retângulo
                cv.circle(cache,(rec[2][0],rec[2][1]),0,(0,0,255),8) # Canto superior direito do retângulo
                cv.circle(cache,(rec[3][0],rec[3][1]),0,(0,0,255),8) # Canto inferior direito do retângulo
                # Desenha o retângulo que envolve a elipse
                box = np.array([[rec[0][0],rec[0][1]],[rec[1][0],rec[1][1]],[rec[2][0],rec[2][1]],[rec[3][0],rec[3][1]]])
                cv.drawContours(cache,[box],0,(0,0,255),1)
                cv.ellipse(cache,(ellipse_center,ellipse_size,ellipse_angle), (0,0,255), 1)
                cv.imshow('Original',cache)
            except:
                None

        
        

    return

## **1.1 Definições de Funções Auxiliares**

In [28]:
def aux_plot_points(v):
    
    fig, ax = plt.subplots()

    for i in range (len(v)):

        x = list()
        y = list()

        for j in range(len(v[i])):
            x.append(frame_width - v[i][j][0][0])
            y.append(frame_height - v[i][j][0][1])
            
        ax.plot(x, y, 'o--' ,color='grey', alpha=0.3)
    
    plt.show()

    #plot_img(x,y,"sem","Sem", "Contorno", frame_height//50, frame_width//50, True, 0, frame_width, 0, frame_height)

    return

In [29]:
def plot_img(x_axis, y_axis, x_title, y_title, plot_title, h,w, axis, xmin, xmax, ymin, ymax, minimum, maximum):
   fig = plt.figure()  # an empty figure with no Axes
   fig, ax = plt.subplots(figsize=(h, w))
   ax.plot(x_axis, y_axis)  # Plot some data on the axes.

   if axis:
      ax.set(xlim=(xmin, xmax), ylim=(ymin, ymax))


   ax.set(xlabel= x_title, ylabel= y_title,
         title= plot_title)
         
   ax.grid() 
   
   for i in range(len(min_list)):

      ax.annotate('MIN', xy=(x_axis[minimum[i]], y_axis[minimum[i]]), arrowprops=dict(facecolor='black', shrink=0.05))
   
   for i in range(len(max_list)):

      ax.annotate('MAX', xy=(x_axis[int(maximum[i])], y_axis[int(maximum[i])]), arrowprops=dict(facecolor='black', shrink=0.05))

In [30]:
def save_video(video_array, video_name):
    out = cv.VideoWriter(video_name,cv.VideoWriter_fourcc('M','J','P','G'), fps, frame_size)
    
    for i in range(len(video_array)):
        out.write(video_array[i])
    out.release()

    return

In [31]:
def distance(x1,y1,x2,y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

In [32]:
def update_line(hl, x,y):
    hl.set_xdata(x)
    hl.set_ydata(y)
    plt.draw()

# **Main Program**

In [33]:
# open video
file = path + folder + name + extension

# Get 3 main parameters trought user's configuration
# Binary treshold = Binary treshold to detect heart's contour
# Axis rotation = Location of the axis to calculate the revolution solid
# Scale = Horizontal scale applied on the contour
binary_treshold, rot_axis_loc, scale, hor_axis = get_filters_parameters(file)

print("Trehold:",binary_treshold)
print("Axis:", rot_axis_loc)
print("Horrizontal Scale:", scale)
print("Horrizontal Axis:", hor_axis)

avg_x_center, avg_y_center = get_average_center(file,binary_treshold)

video = cv.VideoCapture(file)   # Open video

# Test if video was correctly open
if video.isOpened: 
    print("Was open correctly")
    
    # get video fps and number of frames
    fps = int(video.get(cv.CAP_PROP_FPS))
    total_frame = video.get(cv.CAP_PROP_FRAME_COUNT)
    print("Video fps:",fps,"fps")
    
else:
    print("Video wasn't opened corretly", name)
    exit    

Trehold: 247
Axis: 163
Horrizontal Scale: 1.0
Horrizontal Axis: 104
Was open correctly
Video fps: 30 fps


In [34]:
kernel_size = 5

alpha = 0.6
beta = (1.0 - alpha)

method = False

save = False
canny_array = []
superimposed_aray = []
nai = []
ellipse_param = ()

time = 0

time_axis = np.zeros(int(total_frame))
area_axis = np.zeros(int(total_frame))
width_axis = np.zeros(int(total_frame))
height_axis = np.zeros(int(total_frame))
width_axis2 = np.zeros(int(total_frame))
height_axis2 = np.zeros(int(total_frame))
volume_axis = np.zeros(int(total_frame))
volume_axis2 = np.zeros(int(total_frame))
ellipse_area_axis = np.zeros(int(total_frame))
rev_volume_axis = np.zeros(int(total_frame))
rev_area_axis = np.zeros(int(total_frame))

ellipse_backup = np.zeros(int(total_frame),dtype=object)
rec_backup = np.zeros(int(total_frame),dtype=object)
ellipse_vertex_backup = np.zeros(int(total_frame),dtype=object)

end = False
join= False
stop = False

global skip, timer

skip = False
timer = 0

import global_
global_.ellipse_vertex = np.zeros((4,2))

In [35]:
#Loop que passa o video frame a frame, aplicando os filtros necessarios para a aplicação de Canny
while(not end):
      
    if join == False:
      
        ret, frame = video.read()

        # Get current video time in sec 
        video_time = video.get(cv.CAP_PROP_POS_MSEC)/1000

        # Define index of next, current and previous frame
        next_frame = video.get(cv.CAP_PROP_POS_FRAMES)
        current_frame = next_frame - 1
        previous_frame = current_frame - 1
  
        # Creates a copy of unaltered frame
        cache = np.copy(frame)
        
    else:
        frame = np.copy(cache)

    if ret:

        filtered = functions.filters(frame)   # Apply filters on frame

        # Execute on first iteraction
        if time == 0:
            frame_dimentions = frame.shape
            frame_height = frame.shape[0]
            frame_width = frame.shape[1]
            frame_channels = frame.shape[2]
            frame_size = (frame_width,frame_height)      

            cv.namedWindow('Comandos', cv.WINDOW_NORMAL)
            cv.resizeWindow('Comandos', 700, 10)
            
            cv.createTrackbar('Frame:', 'Comandos', int(current_frame), int(total_frame), video_progression) 
                      
            print(frame_size)
            
        # Aplica o filtro binario e detecta a borda do coracao
        _ , binary_img = cv.threshold(filtered,binary_treshold,255,cv.THRESH_BINARY) # 240
        binary_edges = cv.Canny(binary_img, 50, 100, kernel_size, L2gradient= True)

        # Converte edges para opencv e sobrepoe o resultado em cima do frame original
        binary_edges_convert = cv.cvtColor(binary_img,cv.COLOR_GRAY2BGR)
        binary_superiposed = cv.addWeighted(frame, alpha, binary_edges_convert, beta, 0.0)

        # Encaixa a melhor elipse no contortno do coração
        contours, main_contour, ellipse_param, rec, drawing, area, method,stop = fit_ellipse(binary_img,method,join,stop,scale)
        
        if ellipse_backup[int(current_frame)] != 0:
            ellipse_param = ellipse_backup[int(current_frame)]
            ellipse_center = (int(ellipse_param[0][0]),int(ellipse_param[0][1]))
            ellipse_size = (int(ellipse_param[1][0]),int(ellipse_param[1][1]))
            ellipse_angle = int(ellipse_param[2])
            ellipse_param = (ellipse_center,ellipse_size,ellipse_angle)

            rec = rec_backup[int(current_frame)]
            ellipse_vertex = ellipse_vertex_backup[int(current_frame)]

        cv.ellipse(frame, ellipse_param, (90,90,255), 1)    # Draw ellipse on main frame
        
        # Abre janela com o frame original, filtrado, bordas detectadas e sobreposta em cima do frame original 
        cv.imshow('Original',frame)
        cv.imshow('Processado',binary_img)
        cv.imshow('Canny Binario Sobreposto',binary_superiposed)
        cv.imshow('Representacao',drawing)
        #cv.imshow('Informacao',info)

        #if time == 0:
            #cv.createTrackbar('Angulo:', 'Comandos', int(ellipse_param[2]), 179, ellipse_rotation)
        
                   
        #cv.setMouseCallback('Original', modify_elipse,[frame,rec,cache,ellipse_param])
        
                # Aplica a função de interrupção de evento de mouse
        
        while True:
            key = cv.waitKey(10)

            if key != -1 or skip == True:
                break

        if method == False and skip == False:

            while (key != ord('q')) and (key != ord('a')) and (key != ord('j')) and (key != ord('n')) and (key != ord('p')) :
                if stop == False:
                    key = cv.waitKey(0)

            # Se o comando for "quit" ou não houver mais frames no video, entao o loop é finalizado
            if (key == ord('q')) or (not ret):
                end = True
                join = False
            elif key == ord('a'):
                method = True 
                join = False
            elif key == ord('j'):
                join = True
            elif key == ord('n'):
                join= False
                cv.setTrackbarPos('Frame:', 'Comandos',int(next_frame))
            elif key == ord('p'):
                join= False
                cv.setTrackbarPos('Frame:', 'Comandos',int(next_frame))

        else:
            if not ret:
                join = False
                end = True

        if join == False:

            height,width,ellipse_area,volume = get_paramaters(d_pixel)   
            height2,width2,volume2 = get_paramaters2(d_pixel,contours,rot_axis_loc,hor_axis)     
            volume_axis2[int(current_frame)] = volume2
            height_axis2[int(current_frame)] = height2
            width_axis2[int(current_frame)] = width2
            vol_rev, area_rev = get_vol_rev(contours,rot_axis_loc) 

            # Guarda os valores do tempo, area e largura
            time_axis[int(current_frame)] = video_time
            area_axis[int(current_frame)] = area*d_pixel**2
            volume_axis[int(current_frame)] = volume
            height_axis[int(current_frame)] = height
            width_axis[int(current_frame)] = width
            ellipse_area_axis[int(current_frame)] = ellipse_area
            rev_volume_axis[int(current_frame)] = vol_rev
            rev_area_axis[int(current_frame)] = area_rev
            ellipse_backup[int(current_frame)] = ellipse_center,ellipse_size,ellipse_angle
            rec_backup[int(current_frame)] = rec
            
            import global_
            ellipse_vertex_backup[int(current_frame)] = global_.ellipse_vertex
            
            if key == ord('p'):
                if previous_frame >= 0:
                    video.set(cv.CAP_PROP_POS_FRAMES, previous_frame)
                    time = time - 1/fps
                else:
                    previous_frame = 0
                    time = 0
            else:
                time = time + 1/fps

            skip = False      


    else:
        break  
    
video.release()
cv.destroyAllWindows()    

del video

(320, 240)


In [36]:
if (key == ord('q')):
    time_axis = time_axis[0:int(current_frame)]
    area_axis = area_axis[0:int(current_frame)]
    volume_axis2 = volume_axis2[0:int(current_frame)]
    height_axis2 = height_axis2[0:int(current_frame)]
    width_axis2 = width_axis2[0:int(current_frame)]
    ellipse_area_axis = ellipse_area_axis[0:int(current_frame)]
    volume_axis = volume_axis[0:int(current_frame)]
    height_axis = height_axis[0:int(current_frame)]
    width_axis = width_axis[0:int(current_frame)]
    rev_volume_axis = rev_volume_axis[0:int(current_frame)]
    rev_area_axis = rev_area_axis[0:int(current_frame)]

ellipse_vol_const = (1/6)*np.pi*np.average(height_axis)*(width_axis)**2

# Results

In [37]:
results.show_results(weight,time_axis,width_axis,height_axis,width_axis2,height_axis2,
                        area_axis,rev_area_axis,ellipse_area_axis,
                        volume_axis,rev_volume_axis,ellipse_vol_const,volume_axis2,
                        detection_method)
                        
print("Trehold:",binary_treshold)
print("Axis:", rot_axis_loc)
print("Horrizontal Scale:", scale)
print("Horrizontal Axis:", hor_axis)

print("Solid Rev:",np.average(rev_volume_axis) )
print("Elipse EMNC", np.average(volume_axis))
print("Ellipse EMC", np.average(ellipse_vol_const))
print("Manual:",np.average(volume_axis2))

Trehold: 247
Axis: 163
Horrizontal Scale: 1.0
Horrizontal Axis: 104
Solid Rev: 12455888.691259228
Elipse EMNC 13640032.290372672
Ellipse EMC 13491216.71836049
Manual: 11807595.538819876
